In [ ]:
! pip install cmake dlib face-recognition opencv-python numpy

In [ ]:
import cv2
import face_recognition as fr
import numpy as np
import os

## Load and Convert Images

In [ ]:
path = "Images_Test"
images = []
classNames = []
myList = os.listdir(path)
myList.sort()
encodings = []

In [ ]:
for cl in myList:
    curImg = cv2.imread(f'{path}/{cl}')
    images.append(curImg)
    classNames.append(os.path.splitext(cl)[0])

In [ ]:
print(classNames)